In [1]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize
import pypesto.sample as sample
import pypesto.visualize as visualize

import petab
import numpy as np
from scipy.special import gammaln
import pickle

# import to petab
petab_problem = petab.Problem.from_yaml(
    "data/SS_conversion_reaction.yaml")

In [ ]:
def analytical_b(t, a0, b0, k1, k2):
    return (k2 - k2 * np.exp(-(k2 + k1) * t)) / (k2 + k1)

def simulate_model(x, tvec):
    # assign parameters
    k1, k2, _, _ = x
    # define initial conditions
    a0 = 1
    b0 = 0
    # simulate model
    simulation = [analytical_b(t, a0, b0, k1, k2)
                   for t in tvec]
    return simulation

After importing the model, we need to define the objective function. This time we will do it via an external function that will be used then by pyPESTO instead of using the built-in ones.

For numerical reasons we will implement the log likelihood and log prior.

In [ ]:
def log_prior(x):
    """ Log prior function."""
    # assign variables from input x
    offset = x[2]
    precision = x[3]
    
    # evaluate log normal-gamma prior
    l_prior = alpha * np.log(beta) + (alpha - 0.5) * np.log(precision) \
            + 0.5 * (np.log(kappa) - np.log(2) - np.log(np.pi)) - gammaln(alpha) \
            - beta*precision - 0.5*precision*kappa*(offset - mu)**2
    
    return l_prior


def negative_log_posterior(x):
    """ Negative log posterior function."""
    
    offset = x[2]
    precision = x[3]                                                                            
                                                                                
    # experimental data
    data = np.asarray(petab_problem.measurement_df.measurement)
    # time vector
    tvec = np.asarray(petab_problem.measurement_df.time)
    
    n_timepoints = len(tvec)
    
    # simulate model
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = (offset + np.asarray(_simulation))
    
    # evaluate standard log likelihood
    res = data - simulation
    sum_res = np.sum(res**2)
    
    constant = np.log(precision) - np.log(2) - np.log(np.pi)
    
    l_llh = 0.5 * (n_timepoints * constant - precision * sum_res)
    
    # evaluate log normal-gamma prior
    l_prior = log_prior(x)
    
    # return NEGATIVE log posterior (required for pyPESTO)
    return -(l_llh + l_prior)

Now that we have the objective function defined, we need to create a pyPESTO problem.

In [ ]:
def standard_sampling():
    """Creates a pyPESTO problem."""
    objective = pypesto.Objective(fun=negative_log_posterior)
    problem = pypesto.Problem(objective=objective,  # objective function
                              lb=[-5, -5, -np.inf, 0],  # lower bounds
                              ub=[5, 5, np.inf, np.inf],  # upper bounds
                              x_names=['k1', 'k2', 'offset', 'precision'],  # parameter names
                              x_scales=['log', 'log', 'lin', 'lin'])  # parameter scale
    return problem

In [ ]:
### Prior dependent paramters
# center the scaling parameter
mu=0
# std for scaling parameter --> higher = more constrained / lower = more relaxed
alpha= 100
# center the sigma parameter
beta= 0.1
# std for scaling parameter --> higher = more constrained / lower = more relaxed
kappa= 0.01

# create the estimation problem
problem = standard_sampling()

# MCMC chain length
n_samples= 100000

# call the sampler of choice
sampler = sample.AdaptiveMetropolisSampler()

Now we can perform the actual sampling.

In [ ]:
# Define number of runs
runs = 1

save_results = False # for testing just set to False

# Loop over n runs
for n in range(runs):
    # set initial random seed
    np.random.seed(n)
    # perform MCMC sampling
    result = sample.sample(problem, n_samples=n_samples, sampler=sampler,
                           x0=np.array([-1.2741, -0.6160, 0.7826, 3.6839]))
    # calculate effective sample size
    sample.effective_sample_size(result=result)

    # save the results as a pickle object
    if save_results:
        results = [result.sample_result, 'CR_FP', n]
        with open('Results_CR_FP\\result_CR_FP_'+str(n)+'.pickle', 
                  'wb') as result_file:
            pickle.dump(results, result_file)

There are some built-in visualization functions that one can use.

In [ ]:
# Visualize the log posterior trace
ax = visualize.sampling_fval_trace(result, size=(12,5), full_trace=True)
# Visualize the parameter trace
ax = visualize.sampling_parameters_trace(result, use_problem_bounds=False, full_trace=True, size=(12,5))
# Visualize the one-dimensional marginals --> Important!
ax = visualize.sampling_1d_marginals(result, size=(12,5))